In [29]:
import os
import re
import pdfplumber
import pandas as pd
import camelot
from babel.numbers import parse_decimal


In [ ]:
# for filename in os.listdir(pdf_dir):
#     if filename.lower().endswith(".pdf"):
#         pdf_path = os.path.join(pdf_dir, filename)
#         with pdfplumber.open(pdf_path) as pdf:
#             all_text = ""
#             preserved_layout_text = ""
#             body_lines = []
            
#             for page in pdf.pages:
#                 all_text += page.extract_text() + "\n" or ""
#                 # Use preserved_text with extra parameters to preserve spacing
#                 # preserved_text = "" 
#                 # preserved_text += page.extract_text(
#                 #     layout=True,      # Preserves layout including spaces
#                 #     keep_blank_chars=True,  # Keeps blank characters
#                 #     x_tolerance=1,    # Smaller tolerance preserves spacing better
#                 #     y_tolerance=1
#                 # ) or ""
#                 # preserved_layout_text += preserved_text + "\n"
#                 # print(f"-----All text:\n{all_text}\n-----")
#                 # print(f"-----Preserved Layout Text:\n{preserved_layout_text}\n-----")

#                 # BODY TEXT EXTRACTION
#                 lines = all_text.split('\n')
#                 # More precise header pattern that matches your document's header format
#                 header_pattern = re.compile(r'^\*\d{2}B\d+\*$')  # Pattern like "*05B00041981*"
#                 in_header = False
#                 for line in lines:
#                     # Check for header pattern
#                     if header_pattern.match(line.strip()):
#                         in_header = True
#                         continue
#                     # Skip the line immediately after header pattern
#                     if in_header and line.strip().startswith('Order '):
#                         in_header = False
#                         continue
#                     # Skip footer lines
#                     if re.match(r'^Page \d+ of \d+$', line.strip()):
#                         continue
#                     body_lines.append(line)
                
#                 body_text = '\n'.join(body_lines)
#                 # Remove any remaining blank lines at start/end
#                 body_text = body_text.strip()
#                 # print(f"{body_text}")

#                 # #EXTRACT ORDER DETAILS LINES
#                 order_details_lines = []
#                 capture = False
#                 for line in body_text.split('\n'):
#                     if "This document contains certified products. Please see table below for details." in line:
#                         capture = True
#                         continue
#                     if "Applicable Certifications" in line:
#                         capture = False
#                         continue
#                     if capture and line.strip():
#                         order_details_lines.append(line)
                
#                 # Join the order details lines and clean up
#                 order_details_text = '\n'.join(order_details_lines).strip()
#                 print(order_details_text)
                
#                 #Camelot extraction
#                 tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
#                 camelot_tables = []
#                 for table in tables:
#                     camelot_df = table.df
#                     camelot_tables.append(camelot_df)
#                     # print(f"----\n{camelot_df.to_string(index=False)}\n-----")

In [30]:
pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Other Samples"
# pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Main"

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)

        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""           # Raw extracted text from all pages
            all_pages_body = []     # Store processed body text per page

            # Header pattern like "*05B00041981*"
            header_pattern = re.compile(r'^\*\d{2}B\d+\*$')

            # === PAGE LOOP ===
            for page in pdf.pages:
                # Append raw text to all_text (full PDF content, unprocessed)
                page_text = page.extract_text() or ""
                all_text += page_text + "\n"

                # Clean body text for this page
                lines = page_text.split('\n')
                in_header = False
                page_body_lines = []

                for line in lines:
                    if header_pattern.match(line.strip()):
                        in_header = True
                        continue
                    if in_header and line.strip().startswith('Order '):
                        in_header = False
                        continue
                    if re.match(r'^Page \d+ of \d+$', line.strip()):
                        continue
                    page_body_lines.append(line)

                cleaned_body = '\n'.join(page_body_lines).strip()
                all_pages_body.append(cleaned_body)

            # === Combine cleaned page texts ===
            body_text = "\n".join(all_pages_body)

            # === Camelot table extraction (run ONCE per file) ===
            tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
            camelot_tables = [table.df for table in tables]

            # === Extract order details ===
            order_details_lines = []
            capture = False
            last_total_usd_index = None
            lines = body_text.split("\n")

            # Pre-calc last "Total USD" if "Applicable Certifications" not found
            if "Applicable Certifications" not in body_text:
                for i, line in enumerate(lines):
                    if "Total USD" in line:
                        last_total_usd_index = i

            for i, line in enumerate(lines):
                if "This document contains certified products. Please see table below for details." in line:
                    capture = True
                    continue
                
                if not capture and "shipment FOB date" in line:
                    capture = True

                if "Applicable Certifications" in line:
                    capture = False
                    continue

                # Stop continuing if we hit the last "Total USD" line
                if last_total_usd_index is not None and i > last_total_usd_index:
                    capture = False

                if capture and line.strip():
                    order_details_lines.append(line)

            order_details_text = "\n".join(order_details_lines)


            print(f"All Text:\n{all_text}")  # Raw text
            print(f"Body Text:\n{body_text}")  # Cleaned text without headers/footers
            print(f"Order Details:\n{order_details_text}")  # Extracted order details texts  

All Text:
*05B00041688*
Order 05B00041688
Buy-from Vendor No. 9900960321 Order Date 04-11-2025 (MM-DD-YYYY)
Purchaser Julian Frommann
MANAMI FASHIONS LTD.
E-Mail julian.frommann@brands-
KABIRPUR, ASHULIA,
fashion.com
SAVAR,
Dhaka, 1349 Phone No. +49 4181 / 9422 - 293
Dhaka Payment Terms payment after receipt of invoice
Bangladesh Payment Method foreign payment
Transport Method SEA
Shipping Agent Code SPEDITION
Port of Departure CGP
Shipment Date ETD 05-30-2025 (MM-DD-YYYY)
Cost Center LIDL
TC Required
Ship-to Address Agency to Address
BRANDS Fashion GmbH TRITON TEXTILE LIMITED Bangladesh
Aussenlager House No. 20 (1st + 2nd & 3rd Floor)
Am Ring 11 Road No. 99, Gulshan-02
19376 Ruhner Berge OT Zachow Dhaka
Germany Bangladesh
This document contains certified products. Please see table below for details.
LIDL115DE_G Men's polo short sl. slim fit shipment FOB date 30. May 2025 Price 4,81
light blue Customs no.: 6105 1000 00
INTERNAT1
Size XS S M L
Qty 20 80 440 440
Total 980 PIECES
Amount 4

In [ ]:
sizes = []
quantities = []
found_size_table = False

for i, df in enumerate(camelot_tables):
    df = df.fillna("").astype(str).applymap(lambda x: x.strip())

    for r in range(df.shape[0]):
        row_values = [cell.strip() for cell in df.iloc[r].values]

        if row_values[0].lower().startswith("size"):
            # Extract sizes from the rest of the row
            sizes = [s for s in row_values[1:] if s]

            # Next row is quantities
            if r + 1 < df.shape[0]:
                qty_values = [q for q in df.iloc[r + 1, 1:].values if q]
                quantities = [str(parse_decimal(q, locale="de_DE")) for q in qty_values]
                found_size_table = True
                break
    if found_size_table:
        break

if found_size_table:
    print("Sizes:", sizes)
    print("Quantities:", quantities)
else:
    print("No size/qty table found.")


In [27]:
import re
import pandas as pd
from decimal import Decimal

# Make pandas print everything while debugging
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", None)

def _norm(s):
    s = "" if s is None else str(s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def _is_qty_header(s):
    return bool(re.search(r'^(qty|qty\.|quantity|quantities)$', s, flags=re.I))

def _is_size_header(s):
    return bool(re.search(r'^size(s)?\.?$', s, flags=re.I))

def _looks_numeric(s):
    s = _norm(s)
    s = re.sub(r"[^\d,.\-]", "", s)
    return bool(re.search(r"\d", s))

def _parse_de(s):
    s = _norm(s)
    s = re.sub(r"[^\d,.\-]", "", s)
    if not s or not re.search(r"\d", s):
        return "0"
    try:
        return str(parse_decimal(s, locale="de_DE"))
    except Exception:
        s2 = s.replace(".", "").replace(",", ".")
        try:
            return str(Decimal(s2))
        except Exception:
            return s

# Store all found size tables
all_size_tables = []

for i, df in enumerate(camelot_tables):
    print("\n" + "="*80)
    print(f"DEBUG: Camelot Table {i+1}/{len(camelot_tables)} raw shape={df.shape}")
    
    # Normalize table
    df = df.copy().fillna("").astype(str).applymap(_norm)
    df.columns = [f"c{j}" for j in range(df.shape[1])]
    
    print(df.to_string(index=True))
    print("-"*80)
    
    if df.shape[0] > 0:
        print("DEBUG: header row:", list(df.iloc[0].values))
    if df.shape[1] > 0:
        print("DEBUG: first column:", list(df.iloc[:, 0].values))
    
    current_sizes = []
    current_quantities = []
    
    # ------------------------------------------------------------
    # Strategy 1: Horizontal header row contains "Size"
    # ------------------------------------------------------------
    if df.shape[0] >= 1:
        header = list(df.iloc[0].values)
        if any(_is_size_header(x) or "size" in x.lower() for x in header):
            size_col_idx = None
            for idx, h in enumerate(header):
                if _is_size_header(h) or "size" in h.lower():
                    size_col_idx = idx
                    break

            if size_col_idx is not None:
                candidate_sizes = [v for v in df.iloc[0, size_col_idx+1:].values if _norm(v)]
                print(f"DEBUG: Strategy 1 found size row at header; size_col_idx={size_col_idx}")
                print("DEBUG: candidate_sizes:", candidate_sizes)

                qty_row_idx = None
                for r in range(1, min(df.shape[0], 5)):  # Check next 5 rows for Qty
                    if _is_qty_header(df.iat[r, size_col_idx]):
                        qty_row_idx = r
                        break

                if qty_row_idx is not None:
                    candidate_qty = df.iloc[qty_row_idx, size_col_idx+1:size_col_idx+1+len(candidate_sizes)].values
                    quantities = [_parse_de(q) if _norm(q) else "0" for q in candidate_qty]
                    sizes = candidate_sizes[:len(quantities)]
                    
                    print(f"DEBUG: Strategy 1 matched Qty at row={qty_row_idx}")
                    print("DEBUG: sizes:", sizes)
                    print("DEBUG: quantities:", quantities)
                    
                    if len(sizes) > 0:
                        all_size_tables.append((sizes, quantities))
    
    # ------------------------------------------------------------
    # Strategy 2: Any row contains "Size" horizontally
    # ------------------------------------------------------------
    for r in range(df.shape[0]):
        for c in range(df.shape[1]):
            if _is_size_header(df.iat[r, c]) or "size" in df.iat[r, c].lower():
                candidate_sizes = [v for v in df.iloc[r, c+1:].values if _norm(v)]
                print(f"DEBUG: Strategy 2 found 'Size' at (row={r}, col={c})")
                print("DEBUG: candidate_sizes:", candidate_sizes)
                
                # Look for Qty in next 5 rows
                for qr in range(r+1, min(r+6, df.shape[0])):
                    if _is_qty_header(df.iat[qr, c]):
                        candidate_qty = df.iloc[qr, c+1:c+1+len(candidate_sizes)].values
                        quantities = [_parse_de(q) if _norm(q) else "0" for q in candidate_qty]
                        sizes = candidate_sizes[:len(quantities)]
                        
                        print(f"DEBUG: Strategy 2 matched 'Qty' at row={qr}")
                        print("DEBUG: sizes:", sizes)
                        print("DEBUG: quantities:", quantities)
                        
                        if len(sizes) > 0:
                            all_size_tables.append((sizes, quantities))
                        break
    
    # ------------------------------------------------------------
    # Strategy 3: Vertical columns with Size and Qty headers
    # ------------------------------------------------------------
    if df.shape[0] >= 2:
        header = list(df.iloc[0].values)
        size_col = qty_col = None

        for idx, h in enumerate(header):
            if size_col is None and (_is_size_header(h) or "size" in h.lower()):
                size_col = idx
            if qty_col is None and _is_qty_header(h):
                qty_col = idx

        if size_col is not None and qty_col is not None and size_col != qty_col:
            col_sizes = [v for v in df.iloc[1:, size_col].values if _norm(v)]
            col_qty = [_parse_de(q) if _norm(q) else "0" for q in df.iloc[1:, qty_col].values]
            L = min(len(col_sizes), len(col_qty))
            sizes = col_sizes[:L]
            quantities = col_qty[:L]
            
            print(f"DEBUG: Strategy 3 using vertical columns size_col={size_col}, qty_col={qty_col}")
            print("DEBUG: sizes:", sizes)
            print("DEBUG: quantities:", quantities)
            
            if L > 0:
                all_size_tables.append((sizes, quantities))

# Process all found size tables
if all_size_tables:
    print("\nSUCCESS: Found", len(all_size_tables), "size tables:")
    for i, (sizes, quantities) in enumerate(all_size_tables, 1):
        print(f"\nTable {i}:")
        print("Sizes:", sizes)
        print("Quantities:", quantities)
        
        # Create order entries for each size/qty pair
        for size, qty in zip(sizes, quantities):
            orders.append({
                'Order Number': order_number,
                'Unit': unit,
                'Shipment FOB Date': shipment_fob_date,
                'Color': color,
                'Size': size,
                'Qty': qty,
                'Price': price,
                # ... other fields
            })
else:
    print("\nWARNING: Could not find any size/quantity tables.")
    # Fall back to text-based parsing


DEBUG: Camelot Table 1/4 raw shape=(22, 5)
                      c0          c1                 c2                   c3                                c4
0                                                                                                *05B00041688*
1                                                                                            Order 05B00041688
2    Buy-from Vendor No.  9900960321                              Order Date           04-11-2025 (MM-DD-YYYY)
3                                                                  Purchaser                   Julian Frommann
4   MANAMI FASHIONS LTD.                                                                                      
5                                                                     E-Mail           julian.frommann@brands-
6     KABIRPUR, ASHULIA,                                                                                      
7                                                                   

C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\1511933310.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.copy().fillna("").astype(str).applymap(_norm)
C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\1511933310.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.copy().fillna("").astype(str).applymap(_norm)
C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\1511933310.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.copy().fillna("").astype(str).applymap(_norm)
C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\1511933310.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.copy().fillna("").astype(str).applymap(_norm)


NameError: name 'order_number' is not defined

In [24]:
import re
import pandas as pd
from decimal import Decimal

# Make pandas print everything while debugging
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", None)

def _norm(s):
    s = "" if s is None else str(s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

def _is_qty_header(s):
    return bool(re.search(r'^(qty|qty\.|quantity|quantities)$', s, flags=re.I))

def _is_size_header(s):
    return bool(re.search(r'^size(s)?\.?$', s, flags=re.I))

def _looks_numeric(s):
    # Accept numbers like "4.630", "3,120", "215", "1 420"
    s = _norm(s)
    s = re.sub(r"[^\d,.\-]", "", s)
    return bool(re.search(r"\d", s))

def _parse_de(s):
    s = _norm(s)
    # Strip all non-numeric except separators/sign
    s = re.sub(r"[^\d,.\-]", "", s)
    if not s or not re.search(r"\d", s):
        return ""
    try:
        # If you already imported Babel's parse_decimal:
        return str(parse_decimal(s, locale="de_DE"))
    except Exception:
        # Fallback: naive normalization (best-effort)
        # Assume '.' is thousands, ',' is decimal
        s2 = s.replace(".", "").replace(",", ".")
        try:
            return str(Decimal(s2))
        except Exception:
            return s

sizes = []
quantities = []
found_size_table = False

for i, df in enumerate(camelot_tables):
    print("\n" + "="*80)
    print(f"DEBUG: Camelot Table {i+1}/{len(camelot_tables)} raw shape={df.shape}")

    # Normalize to strings and strip whitespace
    df = df.copy()
    df = df.fillna("").astype(str).applymap(_norm)

    # Rename columns to c0, c1, ... for clarity
    df.columns = [f"c{j}" for j in range(df.shape[1])]

    # Print the whole table
    print(df.to_string(index=True))
    print("-"*80)

    # Quick peeks
    if df.shape[0] > 0:
        print("DEBUG: header row:", list(df.iloc[0].values))
    if df.shape[1] > 0:
        print("DEBUG: first column:", list(df.iloc[:, 0].values))

    # ---------------------------------------------------------------------
    # Strategy 1 (A): Horizontal header row contains "Size"
    # ---------------------------------------------------------------------
    if df.shape[0] >= 1:
        header = list(df.iloc[0].values)
        if any(_is_size_header(x) or "size" in x.lower() for x in header):
            size_col_idx = None
            for idx, h in enumerate(header):
                if _is_size_header(h) or "size" in h.lower():
                    size_col_idx = idx
                    break

            if size_col_idx is not None:
                # Sizes are all cells to the right on the header row
                candidate_sizes = [v for v in df.iloc[0, size_col_idx+1:].values if _norm(v)]
                print(f"DEBUG: Strategy 1 found size row at header; size_col_idx={size_col_idx}")
                print("DEBUG: candidate_sizes:", candidate_sizes)

                # Find a row that starts with Qty/Quantity (any column ok)
                qty_row_idx, qty_col_idx = None, None
                for r in range(1, df.shape[0]):
                    for c in range(df.shape[1]):
                        if _is_qty_header(df.iat[r, c]):
                            qty_row_idx, qty_col_idx = r, c
                            break
                    if qty_row_idx is not None:
                        break

                if qty_row_idx is not None:
                    # Quantities: cells to the right of the found qty cell
                    candidate_qty = [v for v in df.iloc[qty_row_idx, qty_col_idx+1:].values if _norm(v)]
                    # Align lengths if misaligned
                    L = min(len(candidate_sizes), len(candidate_qty))
                    sizes = candidate_sizes[:L]
                    quantities = [_parse_de(q) if _looks_numeric(q) else q for q in candidate_qty[:L]]

                    print(f"DEBUG: Strategy 1 matched Qty at row={qty_row_idx}, col={qty_col_idx}")
                    print("DEBUG: sizes:", sizes)
                    print("DEBUG: quantities:", quantities)

                    if L > 0:
                        found_size_table = True
                        break
                else:
                    print("DEBUG: Strategy 1: no row with Qty/Quantity found below the header.")

    if found_size_table:
        break

    # ---------------------------------------------------------------------
    # Strategy 2 (B): Any row contains "Size" horizontally (not just header)
    # ---------------------------------------------------------------------
    size_hit = None
    for r in range(df.shape[0]):
        for c in range(df.shape[1]):
            if _is_size_header(df.iat[r, c]) or "size" in df.iat[r, c].lower():
                size_hit = (r, c)
                break
        if size_hit:
            break

    if size_hit:
        sr, sc = size_hit
        candidate_sizes = [v for v in df.iloc[sr, sc+1:].values if _norm(v)]
        print(f"DEBUG: Strategy 2 found 'Size' at (row={sr}, col={sc})")
        print("DEBUG: candidate_sizes:", candidate_sizes)

        qty_hit = None
        for r in range(sr+1, df.shape[0]):  # typically below the size row
            for c in range(df.shape[1]):
                if _is_qty_header(df.iat[r, c]):
                    qty_hit = (r, c)
                    break
            if qty_hit:
                break

        if qty_hit:
            qr, qc = qty_hit
            candidate_qty = [v for v in df.iloc[qr, qc+1:].values if _norm(v)]
            L = min(len(candidate_sizes), len(candidate_qty))
            sizes = candidate_sizes[:L]
            quantities = [_parse_de(q) if _looks_numeric(q) else q for q in candidate_qty[:L]]

            print(f"DEBUG: Strategy 2 matched 'Qty' at (row={qr}, col={qc})")
            print("DEBUG: sizes:", sizes)
            print("DEBUG: quantities:", quantities)

            if L > 0:
                found_size_table = True
                break
        else:
            print("DEBUG: Strategy 2: no 'Qty/Quantity' found after the 'Size' row.")

    if found_size_table:
        break

    # ---------------------------------------------------------------------
    # Strategy 3 (C): Vertical columns with headers 'Size' and 'Qty/Quantity'
    # ---------------------------------------------------------------------
    if df.shape[0] >= 2:
        header = list(df.iloc[0].values)
        size_col = qty_col = None

        for idx, h in enumerate(header):
            if size_col is None and (_is_size_header(h) or "size" in h.lower()):
                size_col = idx
            if qty_col is None and _is_qty_header(h):
                qty_col = idx

        if size_col is not None and qty_col is not None and size_col != qty_col:
            col_sizes = [v for v in df.iloc[1:, size_col].values if _norm(v)]
            col_qty = [v for v in df.iloc[1:, qty_col].values if _norm(v)]
            L = min(len(col_sizes), len(col_qty))
            sizes = col_sizes[:L]
            quantities = [_parse_de(q) if _looks_numeric(q) else q for q in col_qty[:L]]

            print(f"DEBUG: Strategy 3 using vertical columns size_col={size_col}, qty_col={qty_col}")
            print("DEBUG: sizes:", sizes)
            print("DEBUG: quantities:", quantities)

            if L > 0:
                found_size_table = True
                break

    print(f"DEBUG: No valid size/qty pattern recognized in table {i+1}.")

# Final report (or fallback)
if found_size_table:
    print("\nSUCCESS: Extracted sizes & quantities from Camelot:")
    print("Sizes:", sizes)
    print("Quantities:", quantities)
else:
    print("\nWARNING: Could not find sizes/quantities in Camelot tables.")
    print("Falling back to your original text-based parsing...")
    # (Your original text-based fallback here)



DEBUG: Camelot Table 1/4 raw shape=(22, 5)
                      c0          c1                 c2                   c3                                c4
0                                                                                                *05B00041688*
1                                                                                            Order 05B00041688
2    Buy-from Vendor No.  9900960321                              Order Date           04-11-2025 (MM-DD-YYYY)
3                                                                  Purchaser                   Julian Frommann
4   MANAMI FASHIONS LTD.                                                                                      
5                                                                     E-Mail           julian.frommann@brands-
6     KABIRPUR, ASHULIA,                                                                                      
7                                                                   

C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\2177679018.py:56: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna("").astype(str).applymap(_norm)
C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\2177679018.py:56: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna("").astype(str).applymap(_norm)
C:\Users\Altersense\AppData\Local\Temp\ipykernel_13084\2177679018.py:56: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.fillna("").astype(str).applymap(_norm)


In [ ]:
# Camelot Extractions with Debugging and Fallback
ship_to_address = 'Not found'
agency_to_address = "Not found"
buying_house_address = "Not found"
agency_full_address = "Not found"

print(f"\n=== Camelot Debug ===")
print(f"Total tables found: {len(camelot_tables)}")

found_table = None
found_row = None
found_col = None

# First pass: Try to find the structured address table
for i, df in enumerate(camelot_tables):
    print(f"\nTable {i+1}: Shape {df.shape}")
    print("First 2 rows preview:")
    print(df.head(2))  # Show first 2 rows
    
    if df.shape[1] >= 2:
        first_cell = str(df.iloc[0, 0])
        print(f"First cell content: '{first_cell}'")
        
        if "Ship-to Address" in first_cell:
            print("--> Found address table in standard position")
            ship_to_address = df.iloc[1, 0].strip()
            agency_to_address = df.iloc[1, 1].strip()
            
            buying_house_address = "\n".join(
                [row[0].strip() for row in df.iloc[2:].values if row[0].strip()]
            )
            agency_full_address = "\n".join(
                [row[1].strip() for row in df.iloc[2:].values if row[1].strip()]
            )
            found_table = df
            break

# Second pass: Fallback search through all tables if not found
if found_table is None:
    print("\nStandard address table not found, initiating fallback search...")
    for i, df in enumerate(camelot_tables):
        print(f"\nSearching Table {i+1} for address patterns...")
        
        # Search through all cells for "Ship-to Address"
        for row_idx in range(df.shape[0]):
            for col_idx in range(df.shape[1]):
                cell_value = str(df.iloc[row_idx, col_idx]).strip()
                if "Ship-to Address" in cell_value:
                    print(f"Found 'Ship-to Address' at Table {i+1}, Row {row_idx+1}, Column {col_idx+1}")
                    found_table = df
                    found_row = row_idx
                    found_col = col_idx
                    break
            if found_table is not None:
                break
        if found_table is not None:
            break

# Extract from found position if located
if found_table is not None:
    if found_row is not None:  # Found in fallback search
        print("\nExtracting from fallback position...")
        # Get the column with "Ship-to Address" label
        ship_col = found_col
        
        # Try to find agency column - look for "Agency" in same row
        agency_col = None
        for col_idx in range(found_table.shape[1]):
            cell_value = str(found_table.iloc[found_row, col_idx]).strip()
            if "Agency" in cell_value and col_idx != ship_col:
                agency_col = col_idx
                break
        
        # Extract ship-to address (next row in same column)
        if found_row + 1 < found_table.shape[0]:
            ship_to_address = str(found_table.iloc[found_row + 1, ship_col]).strip()
            
            # Extract buying house address (subsequent rows in same column)
            buying_house_rows = []
            for row_idx in range(found_row + 2, found_table.shape[0]):
                cell_value = str(found_table.iloc[row_idx, ship_col]).strip()
                if cell_value:
                    buying_house_rows.append(cell_value)
            buying_house_address = "\n".join(buying_house_rows)
        
        # Extract agency information if agency column found
        if agency_col is not None:
            if found_row + 1 < found_table.shape[0]:
                agency_to_address = str(found_table.iloc[found_row + 1, agency_col]).strip()
            
            # Extract full agency address
            agency_rows = []
            for row_idx in range(found_row + 2, found_table.shape[0]):
                cell_value = str(found_table.iloc[row_idx, agency_col]).strip()
                if cell_value:
                    agency_rows.append(cell_value)
            agency_full_address = "\n".join(agency_rows)

print("\nFinal Address Values:")
print(f"Ship-to: {ship_to_address}")
print(f"Agency: {agency_to_address}")
print(f"Buying House: {buying_house_address}")
print(f"Full Agency: {agency_full_address}")

In [ ]:
from babel.numbers import parse_decimal

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    # Extract Amount value
    amount_match = re.search(r'Amount\s([\d\.,]+)', block)
    amount = amount_match.group(1) if amount_match else "0"

    # Convert European format to US float
    amount_usd = float(parse_decimal(amount, locale='de_DE'))

    # Store as dict
    orders.append({
        'Style No': style_no,
        'Amount USD': amount_usd
    })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


In [ ]:

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    # Extract Amount value
    amount_match = re.search(r'Amount\s([\d\.,]+)', block)
    amount = amount_match.group(1) if amount_match else "0"

    # Convert European format to US float
    amount_usd = float(parse_decimal(amount, locale='de_DE'))

    # Store as dict
    orders.append({
        'Style No': style_no
    })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


In [ ]:

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    #Extract Customs no.
    customs_no_match = re.search(r'Customs\s+no\.?\s*:\s*([\d ]+)', block)
    if customs_no_match:
        customs_no = customs_no_match.group(1).strip()

    

    # Store as dict
    orders.append({
        'Style No': style_no,
        'Customs No': customs_no
        })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


In [31]:
import camelot


# Extract all tables
camelot_tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')  # or flavor='lattice' if tables have borders

# Loop through tables and print their content
for i, table in enumerate(camelot_tables):
    print(f"\n=== Table {i+1}/{len(camelot_tables)} ===")
    print(f"Shape: {table.shape}")
    print(table.df)  # Pandas DataFrame



=== Table 1/4 ===
Shape: (22, 5)
                       0           1                  2                    3                                 4
0                                                                                                *05B00041688*
1                                                                                           Order  05B00041688
2    Buy-from Vendor No.  9900960321                              Order Date           04-11-2025 (MM-DD-YYYY)
3                                                                  Purchaser                   Julian Frommann
4   MANAMI FASHIONS LTD.                                                                                      
5                                                                     E-Mail           julian.frommann@brands-
6     KABIRPUR, ASHULIA,                                                                                      
7                                                                             